# G2ConfigMgr Reference

More information:

1. [GitHub repository](https://github.com/Senzing/docker-jupyter)
1. [Senzing documentation](http://docs.senzing.com/?python#g2config)

## Table of contents

1. [Prepare environment](#Prepare-environment)
    1. [Helper class for JSON rendering](#Helper-class-for-JSON-rendering)
    1. [System path](#System-path)
    1. [Initialize variables](#Initialize-variables)
    1. [Create configuration object](#Create-configuration-object)
1. [G2ConfigMgr](#G2ConfigMgr)
    1. [G2ConfigMgr Initialization](#G2ConfigMgr-Initialization)
    1. [addConfig](#addConfig)
    1. [getConfigList](#getConfigList)
    1. [getConfig](#getConfig)
    1. [setDefaultConfigID](#setDefaultConfigID)
    1. [getDefaultConfigID](#getDefaultConfigID)

## Prepare environment

In [ ]:
import os
import sys
import json

# For RenderJSON

import uuid
from IPython.display import display_javascript, display_html, display

### Helper class for JSON rendering

A class for pretty-printing JSON.
Not required by Senzing, 
but helps visualize JSON.

In [ ]:
class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        elif isinstance(json_data, bytearray):
            self.json_str = json_data.decode()
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height:100%; width:100%; background-color: LightCyan"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

### System path

Update system path.

In [ ]:
python_path = "{0}/python".format(
    os.environ.get("SENZING_G2_DIR", "/opt/senzing/g2"))
sys.path.append(python_path)

### Initialize variables

Create variables used for G2Config and G2ConfigMgr.

In [ ]:
module_name = 'pyG2EngineForAddRecord'

config_path = os.environ.get("SENZING_ETC_DIR", "/etc/opt/senzing")
support_path = os.environ.get("SENZING_DATA_VERSION_DIR", "/opt/senzing/data")

resource_path = "{0}/resources".format(
    os.environ.get("SENZING_G2_DIR", "/opt/senzing/g2"))

sql_connection = os.environ.get(
    "SENZING_SQL_CONNECTION", "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db")

verbose_logging = False

senzing_config_dictionary = {
    "PIPELINE": {
        "CONFIGPATH": config_path,        
        "SUPPORTPATH": support_path,
        "RESOURCEPATH": resource_path
    },
    "SQL": {
        "CONNECTION": sql_connection,
    }
}

senzing_config_json = json.dumps(senzing_config_dictionary)

### Create configuration object

The G2Config API is used to create a new JSON configuration from the default template.
For more information, 
view the [senzing-G2Config-reference.ipynb](senzing-G2Config-reference.ipynb) notebook.

In [ ]:
from G2Config import G2Config

g2_config = G2Config()
g2_config.initV2(module_name, senzing_config_json, verbose_logging)

# Create configuration from template file.

config_handle = g2_config.create()

# Save JSON to string variable.

response_bytearray = bytearray()
return_code = g2_config.save(config_handle, response_bytearray)
senzing_model_config_json = response_bytearray.decode()

print("Return Code: {0}".format(return_code))
RenderJSON(response_bytearray)

## G2ConfigMgr

The G2ConfigMgr API is used to add specific JSON configurations to the database,
so that they may be shared across remote systems.
Such configurations are added to the database, and a configuration ID is created for each of them.
The system may then be configured with a specific configuratin ID that points to one of those configurations.
That configuration ID will then be the shared global config.

In [ ]:
from G2ConfigMgr import G2ConfigMgr

### G2ConfigMgr Initialization

To start using G2ConfigMgr, you must first create and initialize an instance of the config manager.
This should be done once per process.

Create a new instance of the config manager and assign it to a variable.
Then, call the appropriate initialization method (such as initV2) to initialize the config manager.

During the call, the initialization methods accept the following parameters:

- **module_name:** A short name given to this instance of the G2 engine (i.e. your G2Module object)
- **senzing_config_json:** A JSON document containing system parameters. For more information, see [http://docs.senzing.com/?python](http://docs.senzing.com/?python#initialization-parameters).
- **verbose_logging:** A boolean which enables diagnostic logging - this will print a massive amount of information to stdout (default = False)

Calling these functions will return "0" upon success - useful for error handling.

In [ ]:
g2_configuration_manager = G2ConfigMgr()

return_code = g2_configuration_manager.initV2(
    module_name,
    senzing_config_json,
    verbose_logging)

print("Return Code: {0}".format(return_code))

### addConfig

Use `addConfig()` to add a configuration JSON document to the data repository.

The `addConfig()` function accepts the following parameters as input:

- **senzing_model_config_json:** The configuration JSON document.
- **config_comment:** A free-form string of comments describing the configuration document.
- **config_id_bytearray:** The returned configID for the new config document registered in the data store.

The function returns "0" upon success. 

In [ ]:
config_comment = "Configuration added from G2SetupConfig."
config_id_bytearray = bytearray()

return_code = g2_configuration_manager.addConfig(
    senzing_model_config_json,
    config_comment,
    config_id_bytearray)

print("Return Code: {0}\nConfig ID: {1}".format(return_code, config_id_bytearray.decode()))

###  getConfigList

Use `getConfigList()` to retrieve a list of the configuration JSON documents contained in the data repository.

The function returns "0" upon success. 

In [ ]:
response_bytearray = bytearray()

return_code = g2_configuration_manager.getConfigList(response_bytearray)

print("Return Code: {0}".format(return_code))
RenderJSON(response_bytearray)

### getConfig

Use `getConfig()` to retrieve a specific configuration JSON document from the data repository.

The `getConfig()` function accepts the following parameters as input:

- **config_id_bytearray:** The configID for the config document that you wish to retrieve.

They also have various arguments used to return response documents.

The function returns "0" upon success. 

In [ ]:
response_bytearray = bytearray()

return_code = g2_configuration_manager.getConfig(
    config_id_bytearray,
    response_bytearray)

print("Return Code: {0}".format(return_code))
RenderJSON(response_bytearray)

### setDefaultConfigID

Use `setDefaultConfigID()` to set the default configuration JSON document in the data repository.

The `setDefaultConfigID()` function accepts the following parameters as input:

- **config_id_bytearray:** The configuration ID for a configuration JSON document previously added to the database.

The function returns "0" upon success. 

In [ ]:
return_code = g2_configuration_manager.setDefaultConfigID(config_id_bytearray)

print("Return Code: {0}".format(return_code))

### getDefaultConfigID

Use `getDefaultConfigID()` to retrieve a specific configuration JSON document from the data repository.

The `getDefaultConfigID()` function accepts the following parameters as input:

- **config_id_bytearray:** Returns the configID for the current default configuration, or 0 if none is set.

The function returns "0" upon success. 

In [ ]:
return_code = g2_configuration_manager.getDefaultConfigID(config_id_bytearray)

print("Return Code: {0}\nConfiguration ID: {1}".format(
    return_code,
    config_id_bytearray.decode()))